In [14]:
from pm4py.objects.log.importer.xes import importer as xes_importer
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
import datetime
import random
import time
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining

In [ ]:
#XES TO CSV
xesToCsv_BPI = xes_importer.apply("Datasets/BPI_Challenge_2012.xes")

xesToCsv_listo = []
xesToCsv_dic = {'no': -1}
xesToCsv_prev_attr = 'no'
xesToCsv_counter = 0
xesToCsv_cur_attr = 'no'

for i in range(0, len(xesToCsv_BPI)):

    for j in range(0, len(xesToCsv_BPI[i])):
        xesToCsv_attr_list = list(xesToCsv_BPI[i][j])
    
        for k in range(0, len(xesToCsv_attr_list)):
            xesToCsv_prev_attr = xesToCsv_cur_attr
            xesToCsv_cur_attr = xesToCsv_attr_list[k]

            if xesToCsv_cur_attr not in xesToCsv_listo:
                xesToCsv_value = xesToCsv_dic[xesToCsv_prev_attr] + 1
            
                for key in xesToCsv_dic:
                    if xesToCsv_dic[key] >= xesToCsv_value:
                        xesToCsv_dic[key] += 1
            
                xesToCsv_dic[xesToCsv_cur_attr] = xesToCsv_value
                xesToCsv_listo.insert(xesToCsv_value, xesToCsv_cur_attr)
        
        xesToCsv_cur_attr = 'no'
        
        
xesToCsv_chain = []
xesToCsv_event = []

for i in range(0, len(xesToCsv_BPI)):
    for j in range(0, len(xesToCsv_BPI[i])):
        xesToCsv_chain.append(i)
        xesToCsv_event.append(j)
        
xesToCsv_df_BPI= pd.DataFrame(index=[np.array(xesToCsv_chain), np.array(xesToCsv_event)], columns = xesToCsv_listo)

for i in range(0, len(xesToCsv_BPI)):
    
    for j in range(0, len(xesToCsv_BPI[i])):

        xesToCsv_attr = xesToCsv_BPI[i][j]
        
        for a in xesToCsv_attr:
            xesToCsv_df_BPI.loc[(i, j), a] = xesToCsv_attr[a]
            
            
xesToCsv_df_BPI.to_csv('Datasets/BPI.csv')

xesToCsv_attr_listo = []
xesToCsv_attr_dic = {'no': -1}
xesToCsv_prev_attr = 'no'
xesToCsv_counter = 0
xesToCsv_cur_attr = 'no'

for i in range(0, len(xesToCsv_BPI)):
    xesToCsv_attr_list = list(xesToCsv_BPI[i].attributes)
    
    for k in range(0, len(xesToCsv_attr_list)):
        xesToCsv_prev_attr = xesToCsv_cur_attr
        xesToCsv_cur_attr = xesToCsv_attr_list[k]

        if xesToCsv_cur_attr not in xesToCsv_attr_listo:
            xesToCsv_value = xesToCsv_attr_dic[xesToCsv_prev_attr] + 1
            
            for key in xesToCsv_attr_dic:
                if xesToCsv_attr_dic[key] >= xesToCsv_value:
                    xesToCsv_attr_dic[key] += 1
            
            xesToCsv_attr_dic[xesToCsv_cur_attr] = xesToCsv_value
            xesToCsv_attr_listo.insert(xesToCsv_value, xesToCsv_cur_attr)
    
    xesToCsv_cur_attr = 'no'
    
    
xesToCsv_attr_chain = []

for i in range(0, len(xesToCsv_BPI)):
    xesToCsv_attr_chain.append(i)
    

xesToCsv_df_BPI_attr = pd.DataFrame(index = [np.array(xesToCsv_attr_chain)], columns = xesToCsv_attr_listo)


for i in range(0, len(xesToCsv_BPI)):

    xesToCsv_attr = xesToCsv_BPI[i].attributes
    
    for a in xesToCsv_attr:
        xesToCsv_df_BPI_attr.loc[i, a] = xesToCsv_attr[a]
        
xesToCsv_df_BPI_attr.to_csv('Datasets/BPI_attr.csv')

if 'Unnamed: 0' in xesToCsv_df_BPI.columns:
    xesToCsv_df_BPI = xesToCsv_df_BPI.rename(columns={'Unnamed: 0': 'chain', 'Unnamed: 1': 'event'})
    xesToCsv_df_BPI = xesToCsv_df_BPI.set_index(['chain', 'event'])
    xesToCsv_df_BPI_attr = xesToCsv_df_BPI_attr.rename(columns={'Unnamed: 0': 'chain'})
    xesToCsv_df_BPI_attr = xesToCsv_df_BPI_attr.set_index('chain')


In [16]:
#Data Load

def fix_time(time):
    return datetime.datetime.fromisoformat(time)

def load_data(BPI = 'BPI.csv', BPI_attr = 'BPI_attr.h5',  data2012 = False):
    df_BPI = pd.read_csv(BPI)
    df_BPI_attr = pd.read_csv(BPI_attr)
    
    if 'Unnamed: 0' in df_BPI.columns:
        df_BPI = df_BPI.rename(columns={'Unnamed: 0': 'chain', 'Unnamed: 1': 'event'})
        df_BPI = df_BPI.set_index(['chain', 'event'])
        df_BPI_attr = df_BPI_attr.rename(columns={'Unnamed: 0': 'chain'})
        df_BPI_attr = df_BPI_attr.set_index('chain')
    
    df_BPI['time:timestamp'] = df_BPI['time:timestamp'].apply(fix_time)

    if data2012:
        df_BPI_attr['REG_DATE'] = df_BPI_attr['REG_DATE'].apply(fix_time)
    
    df_BPI['time:weekday'] = [x.weekday() for x in df_BPI['time:timestamp']]
    df_BPI['time:hour'] = [x.hour for x in df_BPI['time:timestamp']]
    return df_BPI, df_BPI_attr

def load_data_xes(data):
    BPI = xes_importer.apply(data)
    
df, df_attr = load_data(BPI = 'Datasets/BPI_2012.csv', BPI_attr = 'Datasets/BPI_attr_2012.csv', data2012 = True)

In [ ]:
#Split Data

splitData_df = df
splitData_df = splitData_df.rename(columns={'Unnamed: 0': 'case_id', 'Unnamed: 1': 'step_number'})
splitData_df['time:timestamp'] = splitData_df['time:timestamp'].apply(fix_time)
    
splitData_df_attr = df_attr
splitData_df_attr = splitData_df_attr.rename(columns={'Unnamed: 0': 'case_id'})
splitData_df_attr['REG_DATE'] = splitData_df_attr['REG_DATE'].apply(fix_time)

splitData_limit_date = datetime.datetime(2012, 2, 3, 1, 1, 1, 633000, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))

#training data_set
splitData_df_train = splitData_df
#df_train = pd.DataFrame(columns = (df.columns))

splitData_listo = []

for i in tqdm(range(0, 13087)):
    if splitData_df[splitData_df['case_id'] == i].iloc[-1]['time:timestamp'] > splitData_limit_date:
        splitData_listo.append(i)

for i in tqdm(range(0, len(splitData_listo))):
    splitData_df_train = splitData_df_train.drop(splitData_df_train[splitData_df_train['case_id'] == splitData_listo[i]].index)

#for i in tqdm(range(0, len(listo))):
    #df_train = df_train.append(df[df['case_id'] == listo[i]])

splitData_df_train = splitData_df_train.reset_index(drop = True)

#testing data_set
splitData_df_test = pd.DataFrame(columns = (splitData_df.columns))
splitData_listo = []

for i in tqdm(range(0, 13087)):
    if splitData_df[splitData_df['case_id'] == i].iloc[0]['time:timestamp'] > splitData_limit_date:
        splitData_listo.append(i)

for i in tqdm(range(0, len(splitData_listo))):
    splitData_df_test = splitData_df_test.append(splitData_df[splitData_df['case_id'] == splitData_listo[i]])

splitData_df_test = splitData_df_test.reset_index(drop = True)

In [4]:
#Baseline

def load_dataset_csv(file_name : str, file_attribute_name : str) -> list:
    #Loading the two datasets. So far we'll only be using the BPI.csv one
    #This also parses the date column
    df = pd.read_csv(file_name)
    df['time:timestamp'] = df['time:timestamp'].apply(fix_time)
    df_attr = pd.read_csv(file_attribute_name)
    return df, df_attr

# The current version of the tool works with the 2012 dataset!!!
baseline_df, baseline_df_attr = load_dataset_csv('Datasets/BPI_2012.csv', 'Datasets/BPI_attr_2012.csv')

#Let us change the column names to the aforementioned
baseline_df.columns = ['case_id', 'step_number', 'org:resource', 'lifecycle:transition',
       'concept:name', 'time:timestamp']

#Erasing all the non-complete actions from the database:
baseline_df = baseline_df[baseline_df['lifecycle:transition'] == 'COMPLETE']

baseline_df = baseline_df.reset_index()

#I used to suppress the output of the cell, but didn't know it also does not save any progress within this cell
#So in order for df to update globally, we can't suppress the output unfortunately
baseline_df.drop('index', axis=1)

def compute_time_difference():
    #Set the time difference column
    #This function is places here because of the erased non-complete actions
    baseline_df['time:time_between'] = baseline_df['time:timestamp'].diff()
    baseline_df.loc[baseline_df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)
    
compute_time_difference()

def parse_timestamp_data():
    # Adds extra time related columns to the dataset to be used later
    # 0:Monday,..., 6:Sunday
    baseline_df['time:weekday'] = [x.weekday() for x in baseline_df['time:timestamp']]
    baseline_df['time:hour'] = [x.hour for x in baseline_df['time:timestamp']]
    
parse_timestamp_data()

def creating_dict_for_next_step_stats (df : pd.DataFrame, concept_name : str) -> dict:
    '''For an input action checks for all the possible next actions and counts their occurence'''
    
    dic_occurrence = {}
    dic_total_time = {}
    ids = list(df['case_id']) + ['editor: last id'] #Otherwise we check i+1-th position that does not exist
    times = list(df['time:time_between']) + [pd.Timedelta(0)] #Otherwise we check i+1-th position that does not exist
    names = df['concept:name']
    df_concept = df[names == concept_name]
    
    for i, row in df_concept.iterrows():
        
        if (ids[i] == ids[i+1]): #an instance of the same case
            
            if (names[i+1] not in dic_occurrence):
                dic_occurrence[names[i+1]] = 1
                dic_total_time[names[i+1]] = times[i+1]
            else:
                dic_occurrence[names[i+1]] += 1
                dic_total_time[names[i+1]] += times[i+1]
                
        else: #the last instance of the case
            
            if ('editor: close_case' not in dic_occurrence):
                dic_occurrence['editor: close_case'] = 1
                dic_total_time['editor: close_case'] = times[i+1]
            else:
                dic_occurrence['editor: close_case'] += 1
                dic_total_time['editor: close_case'] += times[i+1]
    
    #Compute average time
    dic_avg_time = {}
    
    for key in dic_total_time:
        dic_avg_time[key] = dic_total_time[key] / dic_occurrence[key]
        
    return(dic_occurrence, dic_avg_time)

creating_dict_for_next_step_stats(baseline_df, 'A_SUBMITTED')

def choosing_next_action(dic : dict):
    '''Finds the max value of the input dict and returns the key of the max value'''
    
    max_key = max(dic, key=dic.get)
    
    return(max_key)

choosing_next_action({'A_PARTLYSUBMITTED': 910, "wow": 21, "not_wow": 37})

def cycles_shortcut(actions : list, concept_name : str, max_length : int) -> list or bool:
    '''For saving the operating time, we will try to terminate the baseline early if we get into a loop
    max_length is the longest_trace parameter'''
    
    if(concept_name in actions): #the action has already been done
        
        if(actions[-1] == concept_name): #and it's the most recent action (self-loop)
            
            while(len(actions) < max_length): #filling the rest of the list with the current action if we're in a self-loop
                actions.append(concept_name)
        
        else: #it is not the most recent action
            placement = actions.index(concept_name) #locating the index of the "duplicate"
            aid_array = actions[placement:] #copying the values
            print("aid_array = ", aid_array)
            
            actions = actions + [0] * (max_length-len(actions)) #making [x, y, z, x] into [x, y, z, x, 0, 0, 0, ...]
            print("actions = ", actions)
            
            for i in range(placement+1, max_length): #iterating only over all the indices of 0's in actions
                actions[i] = aid_array[(i-placement)%len(aid_array)] #copying the list's values over and over again
        
        return(actions) #This return has to be then the return of the iterated_expected_actions
    
    else:
        return(False)
    
cycles_shortcut([2, 1, 3, 7], 3, 15)

def iterating_expected_actions(df : pd.DataFrame, concept_name : str, n : int) -> list:
    '''concept_name is the starting point (first action)
    n is the length of the longest trace ever observed
    It is stored in lonest_trace but for runtime reasons, use n so far'''
    
    longest_trace = max(df['step_number']) #finding the longest trace in the database (nr of steps)
    #note that we determine this AFTER deleting some rows with uncomplete steps. We should be running this on full df
    
    i = 0
    actions = [concept_name] #Here is the list that will store all the subsequent actions the algorithm decices to perform
    while (i < n): #terminate if we are exceeding the max number of steps
        wow = creating_dict_for_next_step_stats(df, concept_name)[0] #list all possible options
        concept_name = choosing_next_action(wow) #Choose the most commonly used option
        
        if(cycles_shortcut(actions, concept_name, n) != False): #Checks if we are stuck in a loop
            print("we are stuck in a loop")
            return(cycles_shortcut(actions, concept_name, n))
        
        if(concept_name == 'editor: close_case'): #If it is the "terminate" option - terminate
            break
        actions.append(concept_name) #Add the action to the list
        i += 1
    
    actions.append('editor: close_case')
    print('i = ', i, "n = ", n)
    
    return(actions)

def add_expected_events(df : pd.DataFrame) -> list:
    all_events = df['concept:name'].unique()
    next_event_name_dic = {'editor: close_case': 'editor: close_case'}
    next_event_duration_dic = {'editor: close_case': pd.Timedelta(0)}
    
    for event in all_events:
        next_step_stats = creating_dict_for_next_step_stats(df, event)
        wow = next_step_stats[0] #list all possible options
        concept_name = choosing_next_action(wow) #Choose the most commonly used option
        next_event_name_dic[event] = concept_name
        next_event_duration_dic[event] = next_step_stats[1][concept_name]
    
    return next_event_name_dic, next_event_duration_dic

#Get list of all expected next events and the expected time till that next event
baseline_all_expected_events = add_expected_events(baseline_df)
baseline_all_expected_events

#Add column to dataframe with expected next events and times
baseline_df['expect:next_event'] = baseline_df['concept:name'].map(baseline_all_expected_events[0])
baseline_df['expect:next_time'] = baseline_df['concept:name'].map(baseline_all_expected_events[1]) + baseline_df['time:timestamp']

baseline_df.to_csv('BPI_with_predictions.csv')

iterating_expected_actions(baseline_df, 'A_SUBMITTED', 15)

aid_array =  [3, 7]
actions =  [2, 1, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1108: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


we are stuck in a loop


['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag']

In [ ]:
#Baseline Performance Testing

baseline_limits = [50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000]
baseline_runtimes = [0] * len(baseline_limits)

for i in range(0, len(baseline_limits)):
    print(i)
    df_small = baseline_df[:baseline_limits[i]]
    time_start = time.time()
    iterating_expected_actions(df_small, 'A_SUBMITTED', 15)
    time_end = time.time()
    baseline_runtimes[i] = time_end-time_start

baseline_runtimes
    
import matplotlib.pyplot as plt

baseline_wow = plt.figure(figsize=(7.5, 5))
plt.scatter(x = baseline_limits, y=baseline_runtimes, color = '#AB3334')
plt.title('Runtime of the baseline algorithm (log scale)', fontsize = 16)
plt.ylabel('Operating time [log(s)]', fontsize = 13)
plt.xlabel('Input size [log(n)]', fontsize = 13)
plt.xscale("log")
plt.yscale('log')

baseline_wow2 = plt.figure(figsize=(7.5, 5))
plt.scatter(x = baseline_limits, y=baseline_runtimes, color = '#420CDA')
plt.title('Runtime of the baseline algorithm', fontsize = 16)
plt.ylabel('Operating time [s]', fontsize = 13)
plt.xlabel('Input size [n]', fontsize = 13)

baseline_f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,6))

ax1.scatter(x = baseline_limits, y=baseline_runtimes, color = '#420CDA')
ax1.set_title('Runtime of the baseline algorithm', fontsize = 16)
ax1.set_xlabel('Input size [n]', fontsize = 13)
ax1.set_ylabel('Operating time [s]', fontsize = 13)

ax2.scatter(x = baseline_limits, y=baseline_runtimes, color = '#AB3334')
ax2.set_title('Runtime of the baseline algorithm (log scale)', fontsize = 16)
ax2.set_ylabel('Operating time [log(s)]', fontsize = 13)
ax2.set_xlabel('Input size [log(n)]', fontsize = 13)
ax2.set_xscale("log")
ax2.set_yscale('log')

baseline_f

In [ ]:
#Testing the Baseline

testingBaseline_df = df
testingBaseline_df = testingBaseline_df.rename(columns={'Unnamed: 0': 'case_id', 'Unnamed: 1': 'step_number'})
testingBaseline_df['time:timestamp'] = testingBaseline_df['time:timestamp'].apply(fix_time)
    
testingBaseline_df_attr = df_attr
testingBaseline_df_attr = testingBaseline_df_attr.rename(columns={'Unnamed: 0': 'case_id'})
testingBaseline_df_attr['REG_DATE'] = testingBaseline_df_attr['REG_DATE'].apply(fix_time)

testingBaseline_df = testingBaseline_df[testingBaseline_df['lifecycle:transition'] == 'COMPLETE']
testingBaseline_df = testingBaseline_df.reset_index(drop = True)

testingBaseline_df['time:time_between'] = testingBaseline_df['time:timestamp'].diff()
testingBaseline_df.loc[testingBaseline_df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

#amount of total data that is training data:
testingBaseline_train_data = 0.8
#amount of train data that is validation data:
testingBaseline_validation_data = 0.1

testingBaseline_limit_date = testingBaseline_df[testingBaseline_df['case_id'] == round(testingBaseline_df.iloc[-1, 0]*testingBaseline_train_data)].iloc[0]['time:timestamp']
#limit_date = datetime.datetime(2012, 2, 3, 1, 1, 1, 633000, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))) 

#training data_set
testingBaseline_df_train = testingBaseline_df
testingBaseline_listo = []

for i in tqdm(range(0, 13087)):
    if testingBaseline_df[testingBaseline_df['case_id'] == i].iloc[-1]['time:timestamp'] > testingBaseline_limit_date:
        testingBaseline_listo.append(i)

for i in tqdm(range(0, len(testingBaseline_listo))):
    testingBaseline_df_train = testingBaseline_df_train.drop(testingBaseline_df_train[testingBaseline_df_train['case_id']==testingBaseline_listo[i]].index)

testingBaseline_df_train = testingBaseline_df_train.reset_index(drop = True)

#testing data_set
testingBaseline_mask = np.random.rand(len(testingBaseline_df_train)) < testingBaseline_validation_data
testingBaseline_df_validation = testingBaseline_df_train[testingBaseline_mask]
testingBaseline_df_train = testingBaseline_df_train[~testingBaseline_mask]

#testing data_set
testingBaseline_df_test = pd.DataFrame(columns = (testingBaseline_df.columns))
testingBaseline_listo = []

for i in tqdm(range(0, 13087)):
    if testingBaseline_df[testingBaseline_df['case_id'] == i].iloc[0]['time:timestamp'] > testingBaseline_limit_date:
        testingBaseline_listo.append(i)

for i in tqdm(range(0, len(testingBaseline_listo))):
    testingBaseline_df_test = testingBaseline_df_test.append(testingBaseline_df[testingBaseline_df['case_id'] == testingBaseline_listo[i]])

testingBaseline_df_test = testingBaseline_df_test.reset_index(drop = True)

testingBaseline_df_train.to_csv('BPI_2012_train', index=False)

testingBaseline_df_validation.to_csv('BPI_2012_validation')

testingBaseline_df_test.to_csv('BPI_2012_test', index=False)

testingBaseline_df_train = pd.read_csv('BPI_2012_train')
testingBaseline_df_train['time:timestamp'] = testingBaseline_df_train['time:timestamp'].apply(fix_time)
testingBaseline_df_train['time:time_between'] = testingBaseline_df_train['time:timestamp'].diff()
testingBaseline_df_train.loc[testingBaseline_df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

testingBaseline_df_test = pd.read_csv('BPI_2012_test')
testingBaseline_df_test['time:timestamp'] = testingBaseline_df_test['time:timestamp'].apply(fix_time)
testingBaseline_df_test['time:time_between'] = testingBaseline_df_test['time:timestamp'].diff()
testingBaseline_df_test.loc[testingBaseline_df['step_number'] == 0, 'time:time_between'] = pd.Timedelta(0)

def prediction(prev):
    return baseline_all_expected_events[0][prev]

testingBaseline_df_test['predicted'] = testingBaseline_df_test['concept:name'].apply(prediction)

testingBaseline_df_test['correct'] = np.nan

for i in tqdm(range(0, len(testingBaseline_df_test)-1)):
    
    if testingBaseline_df_test.loc[i, 'case_id'] == testingBaseline_df_test.loc[i+1, 'case_id']:
        testingBaseline_df_test.loc[i, 'correct'] = (testingBaseline_df_test.loc[i, 'predicted'] == testingBaseline_df_test.loc[i+1, 'concept:name'])
    
    else:
         testingBaseline_df_test.loc[i, 'correct'] = (testingBaseline_df_test.loc[i, 'predicted'] == 'editor:close_case')
            
list(testingBaseline_df_test['correct']).count(True)/len(list(testingBaseline_df_test['correct']))

def prediction(prev):
    return baseline_all_expected_events[1][prev]

testingBaseline_df_test['time:between_predicted'] = testingBaseline_df_test['concept:name'].apply(prediction)

def time_difference(var):
    return abs((var[0] - var[1]).total_seconds())

testingBaseline_df_test['time:predicton_off'] = testingBaseline_df_test[['time:time_between', 'time:between_predicted']].apply(time_difference, axis=1)

seconds = testingBaseline_df_test['time:predicton_off'].mean()
minutes = seconds/60
hours = minutes/60
days = hours/24
print(str(seconds) + ', ' + str(minutes) + ', ' + str(hours) + ', ' + str(days))

seconds = testingBaseline_df_test['time:predicton_off'].median()
minutes = seconds/60
hours = minutes/60
days = hours/24
print(str(seconds) + ', ' + str(minutes) + ', ' + str(hours) + ', ' + str(days))

145472/60/24

In [18]:
def find_clusters(cluster_df, column_name):
    df = cluster_df.copy()
    df[column_name] = df[column_name].fillna(-1)
    unique_values = df[column_name].unique()
    
    for val in unique_values:
        df[val] = 0
        df.loc[df[column_name] == val, val] = 1
        
    kmeans = KMeans(n_clusters=15)

    df_grouped = df.groupby('case_id')[unique_values].sum()
    prediction = kmeans.fit_predict(df_grouped[unique_values])
    return dict(enumerate(prediction))

df['concept:name cluster'] = df.case_id.map(find_clusters(df, 'concept:name'))
df['org:resource cluster'] = df.case_id.map(find_clusters(df, 'org:resource'))
df

,case_id,step_number,org:resource,lifecycle:transition,concept:name,time:timestamp,time:weekday,time:hour,concept:name cluster,org:resource cluster
0,0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,5,0,6,8
1,0,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,5,0,6,8
2,0,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,5,0,6,8
3,0,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,5,0,6,8
4,0,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,5,11,6,8
...,...,...,...,...,...,...,...,...,...,...
262195,13086,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423000+01:00,2,23,11,0
262196,13086,2,112.0,SCHEDULE,W_Afhandelen leads,2012-02-29 23:52:01.287000+01:00,2,23,11,0
262197,13086,3,11169.0,START,W_Afhandelen leads,2012-03-01 09:26:46.736000+01:00,3,9,11,0
262198,13086,4,11169.0,COMPLETE,A_DECLINED,2012-03-01 09:27:37.118000+01:00,3,9,11,0


In [19]:
df["next_event"] = df["concept:name"]
df.loc[df['step_number'] == 0, 'next_event'] = 'editor:close_case'
df["next_event"] = df["next_event"].shift(-1)
df.loc[len(df) - 1, 'next_event'] = 'editor:close_case'
df.head()

,case_id,step_number,org:resource,lifecycle:transition,concept:name,time:timestamp,time:weekday,time:hour,concept:name cluster,org:resource cluster,next_event
0,0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,5,0,6,8,A_PARTLYSUBMITTED
1,0,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,5,0,6,8,A_PREACCEPTED
2,0,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,5,0,6,8,W_Completeren aanvraag
3,0,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,5,0,6,8,W_Completeren aanvraag
4,0,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,5,11,6,8,A_ACCEPTED


In [20]:
dtc = DecisionTreeClassifier(max_depth=3)
special_df = df[df['concept:name cluster'] == 0]
special_df = special_df[special_df['org:resource cluster'] == 1]
X_dtc = special_df[['concept:name', 'org:resource', 'step_number', 'time:weekday', 'time:hour']].copy()
y_dtc = special_df[['next_event']]
X_dtc

,concept:name,org:resource,step_number,time:weekday,time:hour
21140,A_SUBMITTED,112.0,0,3,14
21141,A_PARTLYSUBMITTED,112.0,1,3,14
21142,W_Afhandelen leads,112.0,2,3,14
21143,W_Afhandelen leads,11122.0,3,3,14
21144,A_PREACCEPTED,11122.0,4,3,14
...,...,...,...,...,...
238454,W_Nabellen offertes,11049.0,31,1,14
238455,W_Valideren aanvraag,10972.0,32,3,18
238456,O_DECLINED,10972.0,33,3,18
238457,A_DECLINED,10972.0,34,3,18
